In [1]:
#os and i/o
import os
import numpy as np
import glob
from os.path import abspath
import csv

#scientific computing
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize
from pandas import DataFrame, Series
import seaborn as sns
import random as rd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import scipy.stats

import nibabel as nib

Makes design files for the ser, loc, and sim experiments

In [2]:
#preliminary stuff
home_dir = '/data/home/iballard/fd/'
data_dir = os.path.abspath('../')

event_dur = 0 #duration for all events

#get subject list
subjects = list(np.loadtxt(data_dir+ '/subjects.txt',str))
print data_dir

/home/iballard/Dropbox/fd


LOCALIZER

In [3]:
##get timing file for object localizer
def load_timing_file(run,time_to_drop):
    loc_file = home_dir + '/loc_timing/script_kidLoc_2Hz_run' + str(run)
    trial_type = []
    time = []
    with open(loc_file) as f:
        for n,line in enumerate(f):
            l = line.strip('\n').split('\t')
            if len(l)==5: #ignore file header and footer
                trial_type.append(int(l[2]))
                time.append(float(l[1]) - time_to_drop)
    return trial_type, time

In [4]:
#trial type dictionary for object localizer
# def trial_type_dict():
#     return {0:'scramble', 1:'face-adult', 2:'face-child', 3:'body', 4:'limb', 5:'car',
#             6:'guitar', 7:'place', 8:'house', 9:'word', 10:'number'}
def trial_type_dict():
    return {0:'scramble', 1:'face', 2:'face', 3:'body', 4:'body', 5:'object',
            6:'object', 7:'place', 8:'place', 9:'character', 10:'character'}

In [5]:
def write_loc_file(design_dir,cb):
    
    time_to_drop = -3
    trial_dict = trial_type_dict()
    duration = 4 #all blocks are 4s
    
    #set up run counterbalance
    if cb==1:
        run_dict = {1: '1', 2:'2'}
    else:
        run_dict = {1: '2', 2:'1'}
    
    #prepare out file
    out_file = design_dir + 'loc.csv'
    if os.path.exists(out_file):
        os.remove(out_file)
    out_f = open(out_file,'w')
    out_f.write('run,condition,onset,duration\n')
    for run in range(1,3):
        trial_type, time = load_timing_file(run,time_to_drop)

        #write out file
        for n,trial in enumerate(trial_type):
            if n > 0 and trial != trial_type[n-1]: #start of a new trial
                if trial_dict[trial] != 'scramble': #not a condition
                    new_line = ','.join([run_dict[run],trial_dict[trial],str(time[n]),str(4)+'\n'])
                    out_f.write(new_line)
    out_f.close()
    return

In [6]:
#write output
for sub in sub_list:
    design_dir = home_dir + 'data/' + sub + '/design/' #make design folder
    if not os.path.exists(design_dir):
        os.makedirs(design_dir)
    
    #write file
    write_loc_file(design_dir, sub_to_cb[sub])

NameError: name 'sub_list' is not defined

Genereic code for both the SIM and SER experiments (processing log files, etc)

In [15]:
##read in timing info and prepare 
timing_info = pd.read_csv(data_dir + '/timing_info_PE_diff.csv', index_col =0)

##correct for missing data
#no run 1 of scanning data for fd_127
timing_info.loc[(timing_info['sub']=='fd_127') & (timing_info['run']==1),'run'] = 0
timing_info.loc[(timing_info['sub']=='fd_127') & (timing_info['run']==2),'run'] = 1
timing_info.loc[(timing_info['sub']=='fd_127') & (timing_info['run']==3),'run'] = 2

#no run 2 of scanning data for sim condition fd_133
timing_info.loc[(timing_info['sub']=='fd_133') & (timing_info['exp']=='sim') & \
                (timing_info['run']==2),'run'] = 0
timing_info.loc[(timing_info['sub']=='fd_133') & (timing_info['exp']=='sim') & \
                (timing_info['run']==3),'run'] = 2



timing_info = timing_info[timing_info['run'] != 0]

##adjust by the numher of frames to drop
time_to_drop = 9.0 #number of seconds of scans to drop
timing_info['onset'] = timing_info['onset'] - time_to_drop

##sort
timing_info = timing_info.sort(['exp','sub','run','condition','onset'])

#add value columns
timing_info['value'] = 1

In [11]:
#write timing files
diff = True
for exp in ['sim','ser']:
    for sub in subjects:
        
        design_dir = home_dir + 'data/' + sub + '/design/'
        if not diff:
            out_file = design_dir + exp + '_V.csv'
        else:
            out_file = design_dir + exp + '_V_diff.csv'

        timing = timing_info[(timing_info['exp']==exp) & (timing_info['sub'] == sub)]

        ##rename fb by positive or negative valence
        timing.loc[timing['feedback_valence']==1,'condition'] = 'feedback_pos'
        timing.loc[timing['feedback_valence']==-1,'condition'] = 'feedback_neg'

        #write out main regressors
        timing.to_csv(out_file,index = False, columns = ['run','condition','onset',
                                             'duration','value'])

        ##fill out parametric regressors
        timing = timing.fillna(0) 
        timing['value'] = timing['V']

        #drop regressors that dont have parametric effects
        timing = timing[timing['condition']!='A'] #no parametric value for A
        timing = timing[timing['condition']!='feedback_pos'] #no parametric value for A
        timing = timing[timing['condition']!='feedback_neg'] #no parametric value for A

        #rename the parametric regressors
        timing.loc[timing['condition']=='b_plus','condition'] = 'b_plus_PE'
        timing.loc[timing['condition']=='b_minus','condition'] = 'b_minus_PE'
        timing.loc[timing['condition']=='c_plus','condition'] = 'c_plus_PE'
        timing.loc[timing['condition']=='c_minus','condition'] = 'c_minus_PE'
        #write out result
        with open(out_file, 'a') as f:
            timing.to_csv(f, header=False,index = False, columns = ['run','condition','onset',
                                                         'duration','value'])

        if diff:                    
            #write out difference regressors
            timing['value'] = timing['V_mb']
            timing.loc[timing['condition']=='b_plus_PE','condition'] = 'b_plus_PE_diff'
            timing.loc[timing['condition']=='b_minus_PE','condition'] = 'b_minus_PE_diff'
            timing.loc[timing['condition']=='c_plus_PE','condition'] = 'c_plus_PE_diff'
            timing.loc[timing['condition']=='c_minus_PE','condition'] = 'c_minus_PE_diff'
            with open(out_file, 'a') as f:
                timing.to_csv(f, header=False,index = False, columns = ['run','condition','onset',
                                                             'duration','value'])

print 'done'

done


In [32]:
print len(subjects)

34


In [28]:
##write timing files for MVPA analysis
#write timing files
mvpa_dict = {'b_plus':'state','b_minus':'not','c_plus':'not','c_minus':'state'}
for exp in ['sim','ser']:
    for sub in subjects:
        
        design_dir = home_dir + 'data/' + sub + '/design/'
        out_file = design_dir + exp + '_state.csv'

        timing = timing_info[(timing_info['exp']==exp) & (timing_info['sub'] == sub)]

        #drop regressors that dont 
        timing = timing[timing['condition']!='A'] #no parametric value for A
        timing = timing[timing['condition']!='feedback'] #no parametric value for A

        #get state values for each condition
        state = map(lambda x: mvpa_dict[x], timing['condition'].values)    
        timing['condition'] = state

        #write to file
        timing = timing.sort(['onset'])
        timing.to_csv(out_file,index = False, columns = ['run','condition','onset',
                                 'duration'])

In [43]:
#write design for PPI analysis
#write timing files
for exp in ['sim','ser']:
    for sub in subjects:
        
        design_dir = home_dir + 'data/' + sub + '/design/'
        out_file = design_dir + exp + '_ppi.csv'

        timing = timing_info[(timing_info['exp']==exp) & (timing_info['sub'] == sub)]

        ##rename fb by positive or negative valence
        timing.loc[timing['feedback_valence']==1,'condition'] = 'feedback_pos'
        timing.loc[timing['feedback_valence']==-1,'condition'] = 'feedback_neg'

        #set contrast and rename vectors
        timing.ix[timing['condition'] == 'b_minus','value'] = -1
        timing.ix[timing['condition'] == 'c_plus','value'] = -1
        timing.loc[timing['condition']=='b_plus','condition'] = 'state'
        timing.loc[timing['condition']=='b_minus','condition'] = 'state'
        timing.loc[timing['condition']=='c_plus','condition'] = 'state'
        timing.loc[timing['condition']=='c_minus','condition'] = 'state'
        
        #write out main regressors
        timing.to_csv(out_file,index = False, columns = ['run','condition','onset',
                                             'duration','value'])
        
        #next write target regressor
        
        #reload data
        timing = timing_info[(timing_info['exp']==exp) & (timing_info['sub'] == sub)]

        #drop regressors that dont have parametric effects
        timing = timing[timing['condition']!='A'] #no parametric value for A
        timing = timing[timing['condition']!='feedback'] #no parametric value for A

        #set contrast and rename vectors
        timing.ix[timing['condition'] == 'b_minus','value'] = -1
        timing.ix[timing['condition'] == 'c_minus','value'] = -1
        timing.loc[timing['condition']=='b_plus','condition'] = 'target'
        timing.loc[timing['condition']=='b_minus','condition'] = 'target'
        timing.loc[timing['condition']=='c_plus','condition'] = 'target'
        timing.loc[timing['condition']=='c_minus','condition'] = 'target'
        with open(out_file, 'a') as f:
            timing.to_csv(f, header=False,index = False, columns = ['run','condition','onset',
                                                         'duration','value'])        
        #write mean regressor
        timing['value'] = 1
        timing.loc[timing['condition']=='target','condition'] = 'mean'
        with open(out_file, 'a') as f:
            timing.to_csv(f, header=False,index = False, columns = ['run','condition','onset',
                                                         'duration','value'])
